In [1]:
import http.server
import socketserver
import threading
import json


import pygame
from datetime import datetime, timedelta
pygame.init()
pygame.mixer.music.load("flight.mp3")

class school_class:
    def __init__(self,start_hour,start_minute,end_hour,end_minute,class_name):
        self.start = datetime(1,1,1,start_hour,start_minute,0,0)
        self.end = datetime(1,1,1,end_hour,end_minute,0,0)
        self.playing = False
        self.in_session = False
        self.name = class_name
        
    def __gt__(self, other):
        return self.name > other.name or self.start > other.start or self.end > other.end
    
    def __lt__(self, other):
        return self.name < other.name or self.start < other.start or self.end < other.end
        
    def __eq__(self, other):
        return self.name == other.name or self.start == other.start or self.end == other.end

    def toJson(self,is_current):
        start_correct_date = datetime.now()
        start_correct_date = start_correct_date.replace(hour=self.start.hour,minute=self.start.minute, second=self.start.second)
        
        end_correct_date = datetime.now()
        end_correct_date = end_correct_date.replace(hour=self.end.hour,minute=self.end.minute,second=self.end.second)
        
        if is_current:
            seconds = (end_correct_date - datetime.now()).seconds
        else:
            seconds = (start_correct_date - datetime.now()).seconds
        minutes = seconds/60
        seconds = seconds % 60
        import math
        if len(str(seconds)) == 1:
            seconds = "0" + str(seconds)
        else:
            seconds = str(seconds)
        time_left = f"{math.floor(minutes)}:{seconds}"
            
        return {"name":self.name, "start":str(start_correct_date),"end":str(end_correct_date),"time_left":time_left}
    
    def is_one_minute_away(self):
        compare_date = datetime.now() + timedelta(minutes=1)
        if(self.playing):
            return False
        if(compare_date.time() > self.start.time() and datetime.now().time() < self.start.time()):
            return True
        else:
            return False
    
    def is_currently_in_class(self):
        return (datetime.now().time() > self.start.time() and datetime.now().time() < self.end.time())
    
    def start_class(self):
        pygame.mixer.music.pause()
        self.in_session = True
        self.playing = False
    
    def end_class(self):
        self.in_session = False
    
    def play(self):
        self.playing = True
        pygame.mixer.music.rewind()
        pygame.mixer.music.play()


#State machi#ne that  controls what the 
#web app shows and when plays music before class
current_class = None
next_class = None
acknowledge_button_pushed = False
def class_loop():
    global current_class, next_class
    incoming_class = None
    import time
    waiting_minute = False
    while not cancelled:

        #setup state for next class info
        temp_next_class = None
        for class_in_list in classes:
            if temp_next_class is None:
                if class_in_list.start.time() > datetime.now().time() :
                    temp_next_class = class_in_list
            else:
                if temp_next_class.start.time() >  class_in_list.start.time() :
                    temp_next_class = class_in_list
            if temp_next_class is not None:
                next_class = temp_next_class
            else:
                next_class = None
    
        #setup state for current class, play song when its one minute away
        if(current_class is None and incoming_class is None):
            for class_in_list in classes:
                if(class_in_list.is_one_minute_away()):
                    class_in_list.play()
                    incoming_class = class_in_list
                    print("class is about to start")
                elif(class_in_list.is_currently_in_class()):
                    incoming_class = class_in_list #will update on the next loop but wont play song so we dont start playing music during consecutive classes
        elif (incoming_class is not None):
            if(incoming_class.is_currently_in_class()):
                print("class is starting")
                incoming_class.start_class()
                current_class = incoming_class
                incoming_class = None
        else:
            if(not current_class.is_currently_in_class()):
                print("class is ending")
                current_class.end_class()
                current_class = None    
        time.sleep(1)
    

#the list of classes
file = open("classes.txt")
data = file.read()
file.close()
classes = []
for line in data.split("\n"):
    parts = line.split(",")
    start = parts[0].split(":")
    end = parts[1].split(":")
    classes.append(school_class(int(start[0]), int(start[1]), int(end[0]), int(end[1]), parts[2] ))



cancelled = False

thread = threading.Thread(target=class_loop)
thread.start()
    

def datetime_converter(o):
    if isinstance(o, datetime.datetime):
        return o.__str__()

class MyHttpRequestHandler(http.server.SimpleHTTPRequestHandler):
    def set_headers_json(self):
        self.send_response(200)
        self.send_header('Content-type', 'application/json')
        self.end_headers()
        
    def do_HEAD(self):
        self._set_headers()
    
    def do_GET(self):
        global current_class, next_class
        #print(self.path)
        if self.path == '/':
            self.path = 'index.html'
            return http.server.SimpleHTTPRequestHandler.do_GET(self)
        elif self.path == "/class_state":
            self.set_headers_json()
            #print("im here")
            self.wfile.write(json.dumps({"current_class": current_class.toJson(True) if current_class is not None else None, "next_class":next_class.toJson(False) if next_class is not None else None }).encode())
            #self.send_response(200)
            
def run_server():
    handler_object = MyHttpRequestHandler
    PORT = 9000
    my_server = socketserver.TCPServer(("", PORT), handler_object)
    # Star the server
    my_server.serve_forever()
    
thread_server = threading.Thread(target=run_server)
thread_server.start()

pygame 1.9.4.post1
Hello from the pygame community. https://www.pygame.org/contribute.html


127.0.0.1 - - [06/Dec/2020 16:25:31] "GET / HTTP/1.1" 304 -
127.0.0.1 - - [06/Dec/2020 16:25:31] "GET /class_state HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2020 16:25:32] "GET /class_state HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2020 16:25:33] "GET /class_state HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2020 16:25:34] "GET /class_state HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2020 16:25:35] "GET /class_state HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2020 16:25:36] "GET /class_state HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2020 16:25:37] "GET /class_state HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2020 16:25:38] "GET /class_state HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2020 16:25:39] "GET /class_state HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2020 16:25:40] "GET /class_state HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2020 16:25:41] "GET /class_state HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2020 16:25:42] "GET /class_state HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2020 16:25:43] "GET /class_state HTTP/1.1" 200 -
127.0.0.1 - - [06

In [3]:
!sudo service school_classes restart

In [13]:
pygame.init()
pygame.mixer.music.load("flight.mp3")
pygame.mixer.music.play()

In [10]:
pygame.mixer.music

<module 'pygame.mixer_music' from '/usr/lib/python3/dist-packages/pygame/mixer_music.cpython-37m-arm-linux-gnueabihf.so'>

class is about to start
class is starting


In [12]:
file = open("classes.txt")
data = file.read()
file.close()
classes = []
for line in data.split("\n"):
    parts = line.split(",")
    start = parts[0].split(":")
    end = parts[1].split(":")
    classes.append(school_class(int(start[0]), int(start[1]), int(end[0]), int(end[1]), parts[2] ))

In [4]:
(datetime.now() + timedelta(days=1)).weekday()

0

10.0.0.23 - - [06/Dec/2020 10:05:56] "GET /class_state HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2020 10:05:56] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 10:05:57] "GET /class_state HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2020 10:05:57] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 10:05:58] "GET /class_state HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2020 10:05:58] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 10:05:59] "GET /class_state HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2020 10:05:59] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 10:06:00] "GET /class_state HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2020 10:06:00] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 10:06:01] "GET /class_state HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2020 10:06:01] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 10:06:02] "GET /class_state HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2020 10:06:02] "GET /class_state HTTP/1.1" 200 -
10.0.0

In [4]:

start_correct_date = datetime.now()
start_correct_date = start_correct_date.replace(hour=next_class.start.hour,minute=next_class.start.minute, second=next_class.start.second)

end_correct_date = datetime.now()
end_correct_date = end_correct_date.replace(hour=next_class.end.hour,minute=next_class.end.minute,second=next_class.end.second)

seconds = (datetime.now() - start_correct_date).seconds
minutes = seconds/60
seconds = seconds % 60
import math
if len(str(seconds)) == 1:
    seconds = "0" + str(seconds)
else:
    seconds = str(seconds)
time_left = f"{math.floor(minutes)}:{seconds}"
time_left    


10.0.0.23 - - [06/Dec/2020 09:57:08] "GET /class_state HTTP/1.1" 200 -


'1397:08'

10.0.0.23 - - [06/Dec/2020 09:57:09] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:57:10] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:57:11] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:57:12] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:57:13] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:57:14] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:57:15] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:57:16] "GET /class_state HTTP/1.1" 200 -


In [4]:
classes_test[0] == classes[1]

False

In [11]:
dir(classes_test[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'end',
 'end_class',
 'in_session',
 'is_currently_in_class',
 'is_one_minute_away',
 'name',
 'play',
 'playing',
 'start',
 'start_class',
 'toJson']

In [12]:
(start_correct_date - datetime.now()).seconds

2444

10.0.0.23 - - [06/Dec/2020 09:59:16] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:59:17] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:59:18] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:59:19] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:59:20] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:59:21] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:59:22] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:59:23] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:59:24] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:59:25] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:59:26] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:59:27] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:59:28] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:59:29] "GET /class_state HTTP/1.1" 200 -
10.0.0

In [10]:
print(start_correct_date)
datetime.now()

2020-12-06 10:40:00.771531


10.0.0.23 - - [06/Dec/2020 09:58:44] "GET /class_state HTTP/1.1" 200 -


datetime.datetime(2020, 12, 6, 9, 58, 44, 873907)

In [11]:
print("1")

1


10.0.0.23 - - [06/Dec/2020 09:58:45] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:58:46] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:58:47] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:58:48] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:58:49] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:58:50] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:58:51] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:58:52] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:58:53] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:58:54] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:58:55] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:58:56] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:58:57] "GET /class_state HTTP/1.1" 200 -
10.0.0.23 - - [06/Dec/2020 09:58:58] "GET /class_state HTTP/1.1" 200 -
10.0.0

In [78]:


json.dumps({"current_class": current_class.toJson() if current_class is not None else None, "next_class":next_class.toJson() if next_class is not None else None })

'{"current_class": {"name": "Morning Meeting", "start": "2020-12-06 07:05:00.906700", "end": "2020-12-06 07:07:00.906796"}, "next_class": null}'

/class_state
im here
/favicon.ico


10.0.0.23 - - [06/Dec/2020 07:51:51] "GET /class_state HTTP/1.1" 200 -


/
/favicon.ico


10.0.0.23 - - [06/Dec/2020 07:55:30] "GET / HTTP/1.1" 200 -


...

10.0.0.23 - - [06/Dec/2020 08:37:16] "GET /class_state HTTP/1.1" 200 -


/class_state
im here


10.0.0.23 - - [06/Dec/2020 08:37:26] "GET /class_state HTTP/1.1" 200 -


/class_state
im here


10.0.0.23 - - [06/Dec/2020 08:37:36] "GET /class_state HTTP/1.1" 200 -


In [77]:
current_class = classes[0]

In [2]:
from datetime import datetime, timedelta

now = datetime.now()

In [2]:
timedelta(minutes=1)

datetime.timedelta(seconds=60)

/class_state
im here
/favicon.ico


10.0.0.23 - - [06/Dec/2020 07:51:27] "GET /class_state HTTP/1.1" 200 -


In [79]:
start = datetime.now()

/class_state
im here


10.0.0.23 - - [06/Dec/2020 08:37:46] "GET /class_state HTTP/1.1" 200 -


In [93]:
seconds = 100
minutes = seconds/60
seconds = seconds % 60
import math
if len(str(seconds)) == 1:
    seconds = "0" + str(seconds)
else:
    seconds = str(seconds)
time_left = f"{math.floor(minutes)}:{seconds}"
time_left    

'1:40'

/class_state
im here


10.0.0.23 - - [06/Dec/2020 08:48:56] "GET /class_state HTTP/1.1" 200 -


/class_state
im here


10.0.0.23 - - [06/Dec/2020 08:49:06] "GET /class_state HTTP/1.1" 200 -


...

10.0.0.23 - - [06/Dec/2020 08:55:16] "GET /class_state HTTP/1.1" 200 -


/class_state
im here


10.0.0.23 - - [06/Dec/2020 08:55:26] "GET /class_state HTTP/1.1" 200 -


/class_state
im here


10.0.0.23 - - [06/Dec/2020 08:55:36] "GET /class_state HTTP/1.1" 200 -


In [87]:
elapsed = end - start
elapsed.seconds

64

/class_state
im here


10.0.0.23 - - [06/Dec/2020 08:40:36] "GET /class_state HTTP/1.1" 200 -


/class_state
im here


10.0.0.23 - - [06/Dec/2020 08:40:46] "GET /class_state HTTP/1.1" 200 -


...

10.0.0.23 - - [06/Dec/2020 08:47:16] "GET /class_state HTTP/1.1" 200 -


/class_state
im here


10.0.0.23 - - [06/Dec/2020 08:47:26] "GET /class_state HTTP/1.1" 200 -


/class_state
im here


10.0.0.23 - - [06/Dec/2020 08:47:36] "GET /class_state HTTP/1.1" 200 -


In [26]:
cancelled = True
thread.join()

class is about to start
checking start
07:05:00
07:04:16.898142
07:07:00
checking start
07:05:00
07:04:17.900680
07:07:00
checking start
07:05:00
07:04:18.902874
07:07:00
checking start
07:05:00
07:04:19.904975
07:07:00
checking start
07:05:00
07:04:20.907202
07:07:00
checking start
07:05:00
07:04:21.911982
07:07:00
checking start
07:05:00
07:04:22.913854
07:07:00
checking start
07:05:00
07:04:23.915845
07:07:00
checking start
07:05:00
07:04:24.917871
07:07:00
checking start
07:05:00
07:04:25.920072
07:07:00
checking start
07:05:00
07:04:26.922227
07:07:00
checking start
07:05:00
07:04:27.924285
07:07:00
checking start
07:05:00
07:04:28.929500
07:07:00
checking start
07:05:00
07:04:29.930898
07:07:00
checking start
07:05:00
07:04:30.932211
07:07:00
checking start
07:05:00
07:04:31.934391
07:07:00
checking start
07:05:00
07:04:32.936568
07:07:00
checking start
07:05:00
07:04:33.938585
07:07:00
checking start
07:05:00
07:04:34.940712
07:07:00
checking start
07:05:00
07:04:35.942134
07:07

In [23]:
datetime.now()

datetime.datetime(2020, 12, 6, 7, 1, 5, 400559)

In [22]:
cancelled = True

In [4]:
for current_class in classes:
    print(current_class.is_currently_in_class())

False
False
False
False
False
False
True


In [66]:
datetime.now().minute

46

In [3]:
x = (1,2,3)

In [6]:
x[2]

3

In [1]:

while pygame.mixer.music.get_busy() == True:
    continue

pygame 1.9.4.post1
Hello from the pygame community. https://www.pygame.org/contribute.html


KeyboardInterrupt: 